# Correctness Test

## Set up

In [1]:
import numpy as np
import time
from neo4jConnector import startNeo4jSession

In [2]:
driver, session = startNeo4jSession(port=7688)

In [3]:
## parameter settings

# original set
b = 'Block'
t = 'Transaction'
a = 'Address'

# test set
bt = b + '_test'
tt = t + '_test'
at = a + '_test'

# analysis parameters
blockClasses       = [b, bt]
transactionClasses = [t, tt]
addressClasses     = [a, at]
start_block_height = 1
start_block_height = 164467

In [4]:
'''##  test settings
height=1
txid = '60a20bd93aa49ab4b28d514ec10b06e1829ce6818ec06cd3aabd013ebcdc4bb1'
blockClass = b
txClass = t
addrClass = a'''

"##  test settings\nheight=1\ntxid = '60a20bd93aa49ab4b28d514ec10b06e1829ce6818ec06cd3aabd013ebcdc4bb1'\nblockClass = b\ntxClass = t\naddrClass = a"

## Analysis

### Functions

#### Queries

In [5]:
### Blocks
## properties
def getBlockPropertiesByHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}}) RETURN b'#.format(blockClass, height)

## edges
def getBlockEdgesByHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}})<-[r]-() RETURN r'


###  Transactions
## retrieve transactions
def getTransactionsFromBlockHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}})<-[:BELONGS_TO]-(t) RETURN t.txid'

## TR properties
#def getTransactionPropertiesByTxidQuery(txClass, txid):
#    return f'MATCH (t:{txClass} {{txid: \"{txid}\"}}) RETURN t'

# alternative version
def getTransactionPropertiesByTxidQuery(txClass, txid):
    return f'MATCH (t:{txClass} {{txid: \"{txid}\"}}) RETURN t.txid, t.inDegree, t.outDegree, t.inSum, t.outSum, t.date'

### Addresses
## retrieve addresses

# alternative without addressclass
def getAddressFromTransactionIDQuery(txClass, txid):
    #return f'MATCH (t:{txClass} {{txid: \"{txid}\"}}) WHERE (t)<-[:SENDS]-(a) OR (t)-[:RECEIVES]->(a) RETURN a'
    return f'''
    MATCH (t:{txClass} {{txid: \"{txid}\"}}) WITH t MATCH (a)-[:SENDS]->(t)    RETURN a.address as address
    UNION
    MATCH (t:{txClass} {{txid: \"{txid}\"}}) WITH t MATCH (a)<-[:RECEIVES]-(t) RETURN a.address as address
    '''


def getSendsEdgeFromTransactionIDQuery(txClass, txid):
    return f'''
    MATCH (t:{txClass} {{txid: \"{txid}\"}})<-[s:SENDS]-(a) RETURN s.value, a.address
    '''

def getReceivesEdgeFromTransactionIDQuery(txClass, txid):
    return f'''
    MATCH (t:{txClass} {{txid: \"{txid}\"}})-[r:RECEIVES]-(a) RETURN r.value, a.address
    '''


## properties  
def getAddressPropertiesByAddressQuery(addrClass, address):
    return f'MATCH (a:{addrClass} {{address: \"{address}\"}}) RETURN a'

# todo: match properties of edges

#### Other functions

In [6]:
def runQuery(query):
#    return session.run(query).data()
    return session.run(query)#.values()


## second version 
def matchProperties(nodeClassList, 
                    query,
                    returnType, 
                    node_identifier, 
                    match_count, 
                    mismatches, 
                    matchAddressProperties=False):
    
    outputs = []
    
    #loop through different labels of the nodes
    for nodeClass in nodeClassList:
        #send query to neo4j
        #print(query(nodeClass, node_identifier))
        results = runQuery(query(nodeClass, node_identifier))
        
        
        ## strip off data
        # data for properties of single node
        if returnType == 'data':
            results = results.data()
            
        # values for list of one property
        elif returnType == 'values':
            results = results.values()
            
        # parameter missing
        else:
            raise Exception('returnType not set')
        
        outputs.append(results)
    
    
    #print(outputs[0])
        
    if len(outputs) != 2:
        raise Exception('Length of query output does not match.')

        
    # check if equivalent: 
    ## todo: - extend to comparison of arbitrary many 
    ##       - extend for addresses (differing in coinbase and in/outdegree)

    #outputs arre the same
    if compareOutputs(outputs, returnType) == True: 
        match_count += 1

    else:
        # todo: extend outputs to dictionary containing all relevant characters
        extended_outputs                              = {}
        extended_outputs['query']                     = query
        extended_outputs['node_identifier']           = node_identifier
        extended_outputs[nodeClassList[0]+'_outputs'] = outputs[0]
        extended_outputs[nodeClassList[1]+'_outputs'] = outputs[1]
        
        mismatches.append(extended_outputs)

    return match_count, mismatches, outputs


def compareOutputs(outputs, returnType):
    
    #outputs = [v for k,v in outputs]
    #print(outputs)
    match = False 
    o1 = outputs[0]
    o2 = outputs[1]
    
    if returnType == 'values':
        if np.all(np.isin(o1, o2)) and np.all(np.isin(o2, o1)):
            match = True
    
    elif o1 == o2:
        match = True
        
    return match




### Correctnes check

In [7]:
# overall parameter
mismatches  = []
match_count = 0 

#timing
startTime = time.time()

#loop through blocks
for block_height in np.arange(start_block_height, start_block_height+1):
    
    # check whether returned block properties match
    match_count, mismatches, outputs = matchProperties(nodeClassList = blockClasses, 
                                                       query = getBlockPropertiesByHeightQuery,
                                                       returnType = 'data',
                                                       node_identifier = block_height,
                                                       match_count = match_count,
                                                       mismatches = mismatches)
    

    # check whether returned txids  match
    match_count, mismatches, outputs = matchProperties(nodeClassList = blockClasses, 
                                                       query = getTransactionsFromBlockHeightQuery,
                                                       returnType = 'values',
                                                       node_identifier = block_height,
                                                       match_count = match_count,
                                                       mismatches = mismatches)
     
    # strip off outputs for 
    txids = []
    for tx_list in outputs:
        for txid in tx_list:
            txids.append(txid)
    # deduplicate list. 
    # -> if one label returns a wrong txid, it will be searched for this in both labels
    txids = np.unique(txids)
    
    # start loop for all txids for that block
    for txid in txids:

        
        # Match tx properties
        match_count, mismatches, outputs = matchProperties(nodeClassList = transactionClasses, 
                                                           query = getTransactionPropertiesByTxidQuery,
                                                           returnType = 'data',
                                                           node_identifier = txid, 
                                                           match_count = match_count,
                                                           mismatches = mismatches)
        
        match_count, mismatches, outputs = matchProperties(nodeClassList = transactionClasses, 
                                                           query = getSendsEdgeFromTransactionIDQuery,
                                                           returnType = 'values',
                                                           node_identifier = txid, 
                                                           match_count = match_count,
                                                           mismatches = mismatches)


        match_count, mismatches, outputs = matchProperties(nodeClassList = transactionClasses, 
                                                           query = getReceivesEdgeFromTransactionIDQuery,
                                                           returnType = 'data',
                                                           node_identifier = txid, 
                                                           match_count = match_count,
                                                           mismatches = mismatches)
        
        
        # retrieve and match addresses
'''        match_count, mismatches, outputs = matchProperties(nodeClassList = transactionClasses, 
                                                           query = getAddressFromTransactionIDQuery,
                                                           returnType = 'values',
                                                           node_identifier = txid, 
                                                           match_count = match_count,
                                                           mismatches = mismatches)'''

        
        #### Todo: loop through edges to addresses and check properties

        
endTime = np.round((time.time() - startTime),2)
print(f'Retrieval execution time in {endTime} seconds.')
print('Retrieved blocks: ', block_height - start_block_height)
print(f'\nMatches: {match_count}; Mismatches: {len(mismatches)}')
mismatches

Retrieval execution time in 5.32 seconds.
Retrieved blocks:  0

Matches: 60; Mismatches: 20


[{'query': <function __main__.getTransactionsFromBlockHeightQuery(blockClass, height)>,
  'node_identifier': 164467,
  'Block_outputs': [['fe57524da7d70bbf75162620f67208325dbb6d64e362b1d8ed9db67da2e6b33f'],
   ['f58f101f30ffb4fa2396ece4bc62fb391abeb5fd7bd6a2d4d6d0d8ee0f1183a3'],
   ['f4a2bc5b25b3cc088e4ec496463f71b55c5f1ae6e7a5b97566917c8232ab2ae1'],
   ['e9ff9496cd927d9df9e465aec1cafee09cc7f672b8da2d0a13a89e024c03ea95'],
   ['e9b79d86192a4f5662b4bae107ba84bddcc6397cadb9ee2b878e70319261274b'],
   ['d88fbb53146b8662b9e3d58f245d5d7ae0a841e8eb1ad13e8e3e8504a0d8fcea'],
   ['d4958047b8a22da1b4571eb2bdf11804e634ce00b758cc02e27c9114c3a2a0eb'],
   ['cf058352522656c2ce0db5e2ac95b33c68802db3af8fa73a0ad2d6973d41717d'],
   ['cd76d6e8fb462577794493e920f08ca122f75f308938d51d1330555975728f71'],
   ['ca204982176ed3ca1a98fbdb9896527d4ffa7d01e4b63fe11145dbc352b27c2c'],
   ['bb7ae5b89a89b7994ad3a9cedcb4f41db5ff61be5202e9b369e9732f27ae82b8'],
   ['af300c5b191e17ad37aff7aa23c3e9e60f5766ebf21f2496b11332140f

## End connection

In [8]:
session.close()
driver.close()